In [ ]:
!pip install mediapipe opencv-python

In [40]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [41]:
#joint connections based of figure above
mp_pose.POSE_CONNECTIONS

frozenset({(0, 1),
           (0, 4),
           (1, 2),
           (2, 3),
           (3, 7),
           (4, 5),
           (5, 6),
           (6, 8),
           (9, 10),
           (11, 12),
           (11, 13),
           (11, 23),
           (12, 14),
           (12, 24),
           (13, 15),
           (14, 16),
           (15, 17),
           (15, 19),
           (15, 21),
           (16, 18),
           (16, 20),
           (16, 22),
           (17, 19),
           (18, 20),
           (23, 24),
           (23, 25),
           (24, 26),
           (25, 27),
           (26, 28),
           (27, 29),
           (27, 31),
           (28, 30),
           (28, 32),
           (29, 31),
           (30, 32)})

In [42]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    #angle between x and y positioning of limbs between given joints
    angle_ab = np.arctan2(a[1]-b[1], a[0]-b[0])
    angle_bc = np.arctan2(c[1]-b[1], c[0]-b[0])
    
    radians = angle_bc - angle_ab
    
    #convert angle to degrees
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [74]:
def boxing_trainer():
    cap = cv2.VideoCapture(0)

    # Curl counter variables
    jab_counter = 0 
    jab_stage = None
    cross_counter = 0 
    cross_stage = None
    rhook_counter = 0 
    rhook_stage = None
    lhook_counter = 0 
    lhook_stage = None


    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
        
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
      
            # Make detection
            results = pose.process(image)
    
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
            # Extract landmarks (positioning of each joint)
            try:
                landmarks = results.pose_landmarks.landmark
            
                # Get left coordinates
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                # Get right coordinates
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
                # Calculate jab angles
                jab_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                # Jab counter logic
                if (jab_angle < 90):
                    jab_stage = "down"
                if (jab_angle > 150) and (jab_stage =='down'):
                    jab_stage="up"
                    jab_counter +=1
                
                # Calculate cross angles
                cross_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
                # Cross counter logic
                if (cross_angle < 90):
                    cross_stage = "down"
                if (cross_angle > 150) and (cross_stage =='down'):
                    cross_stage="up"
                    cross_counter +=1
                
                # Calculate lhook angles
                lhook_angle_1 = calculate_angle(left_hip, left_shoulder, left_elbow)
                lhook_angle_2 = calculate_angle(left_shoulder, left_elbow, left_wrist)
                # Cross counter logic
                if (lhook_angle_1 < 90):
                    lhook_stage = "down"
                if (lhook_angle_1 > 90) and (lhook_angle_2 < 90) and (lhook_stage =='down'):
                    lhook_stage="up"
                    lhook_counter +=1
                
                # Calculate rhook angles
                rhook_angle_1 = calculate_angle(right_hip, right_shoulder, right_elbow)
                rhook_angle_2 = calculate_angle(right_shoulder, right_elbow, right_wrist)
                # Cross counter logic
                if (rhook_angle_1 < 90):
                    rhook_stage = "down"
                if (rhook_angle_1 > 90) and (rhook_angle_2 < 90) and (rhook_stage =='down'):
                    rhook_stage="up"
                    rhook_counter +=1
                       
            except:
                pass
        
            # Render jab counter
            # Setup status box
            cv2.rectangle(image, (0,0), (100,73), (245,117,16), -1)
            # Rep data
            cv2.putText(image, 'Jab', (15,20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, 'x' + str(jab_counter), 
                        (10,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        
            # Render cross counter
            # Setup status box
            cv2.rectangle(image, (0,73), (100,146), (245,117,16), -1)
            # Rep data
            cv2.putText(image, 'Cross', (15,93), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, 'x' + str(cross_counter), 
                        (10,133), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        
            # Render Left Hook counter
            # Setup status box
            cv2.rectangle(image, (0,146), (100,219), (245,117,16), -1)
            # Rep data
            cv2.putText(image, 'LHook', (15,166), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, 'x' + str(lhook_counter), 
                        (10,206), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        
            # Render Right Hook counter
            # Setup status box
            cv2.rectangle(image, (0,219), (100,292), (245,117,16), -1)
            # Rep data
            cv2.putText(image, 'RHook', (15,239), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, 'x' + str(rhook_counter), 
                        (10,279), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)

        
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                     )               
        
            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

In [75]:
# Run Trainer here
boxing_trainer()